In [1]:
import argparse
import os
import shutil
import time
import pandas as pd
import resnet
import matplotlib.pyplot as plt

import time
import numpy as np
import pandas as pd
import cv2
import PIL.Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

model_names = sorted(name for name in resnet.__dict__
    if name.islower() and not name.startswith("__")
                     and name.startswith("resnet")
                     and callable(resnet.__dict__[name]))

DATA_DIR = "train"

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from module import ActFn, Conv2d, Linear
from torch.autograd import Variable

__all__ = ['ResNet', 'resnet20', 'resnet32', 'resnet44', 'resnet56', 'resnet110', 'resnet1202']

def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, k=8, expansion=1):
        super(BasicBlock, self).__init__()
        self.k = k
        self.expansion = expansion
        # self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv1 = Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False, bitwidth = k)
        self.bn1 = nn.BatchNorm2d(planes)
        self.alpha1 = nn.Parameter(torch.tensor(10.))
        # self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv2 = Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False, bitwidth = k)
        self.bn2 = nn.BatchNorm2d(planes)
        self.alpha2 = nn.Parameter(torch.tensor(10.))
        self.ActFn = ActFn.apply
        
        if stride != 1 or in_planes != planes:
              # original resnet shortcut
              self.shortcut = nn.Sequential(
                    # nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                    Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                    nn.BatchNorm2d(self.expansion * planes)
              )
        else: # nothing done if stride or inplanes do not differ
          self.shortcut = nn.Sequential()

    def forward(self, x):
        # out = F.relu(self.bn1(self.conv1(x)))
        out = self.ActFn(self.bn1(self.conv1(x)), self.alpha1, self.k)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        # out = F.relu(out)
        out = self.ActFn(out, self.alpha2, self.k)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, K=8):
        super(ResNet, self).__init__()
        self.in_planes = 16
        self.k = K

        # self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv1 = Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False, bitwidth = 8)
        self.bn1 = nn.BatchNorm2d(16)
        self.alpha1 = nn.Parameter(torch.tensor(10.))
        self.ActFn = ActFn.apply
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1, expansion=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2, expansion=1)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2, expansion=1)

        # self.linear = nn.Linear(64, num_classes)
        self.linear = Linear(64, num_classes, bitwidth = 8)
        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride, expansion):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, self.k, expansion))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        # out = F.relu(self.bn1(self.conv1(x)))
        out1 = self.ActFn(self.bn1(self.conv1(x)), self.alpha1, self.k)
        out2 = self.layer1(out1)
        out3 = self.layer2(out2)
        out4 = self.layer3(out3)
        out = F.avg_pool2d(out4, out4.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out, out1, out2, out3, out4


def resnet20(k=8):
    print("bit width:", k)
    return ResNet(BasicBlock, [3, 3, 3], K=k)

def test(net):
    import numpy as np
    total_params = 0

    for x in filter(lambda p: p.requires_grad, net.parameters()):
        total_params += np.prod(x.data.numpy().shape)
    print("Total number of params", total_params)
    print("Total layers", len(list(filter(lambda p: p.requires_grad and len(p.data.size())>1, net.parameters()))))

# Check outputs!

In [31]:
# check activations
bits = 3
model = resnet20(bits)
model = model.cuda().eval()
out, out2, out3, out4, out5 = model.forward(torch.rand([1,3,32,32]).cuda())
print("alpha:", model.alpha1)
print(model.alpha1/(np.power(2, bits)-1))
np.unique(out2.cpu().detach().numpy())

bit width: 3
alpha: Parameter containing:
tensor(10., device='cuda:0', requires_grad=True)
tensor(1.4286, device='cuda:0', grad_fn=<DivBackward0>)


array([0.       , 1.4285715, 2.857143 ], dtype=float32)

In [32]:
# check weights
weights = model.state_dict()['conv1.weight'].cpu().detach()
# quantize
from module import DoReFaQuant
quantize = DoReFaQuant.apply
np.unique(quantize(weights, 3).numpy())

array([-1.        , -0.71428573, -0.4285714 , -0.14285713,  0.1428572 ,
        0.42857146,  0.71428573,  1.        ], dtype=float32)

weights are quantized on-the-fly

# Load trained weights